<a href="https://colab.research.google.com/github/kalunkuo/Object-Detection-Tensorflow/blob/main/LCAU_tensorflow_objectDetection_colorLayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **LCAU Object Detection and Image Processing Pipeline**



This code performs object detection on a set of images using a TensorFlow Hub model, draws bounding boxes around detected objects, and saves the annotated images to a specified directory.

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
!pwd

/content/gdrive/MyDrive/0_work/3_MIT/data_collection/18_segments/05_extra_frames/00_selected_frames


In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont, ImageColor, ImageOps
import tensorflow as tf
import tensorflow_hub as hub
from os import listdir
from os.path import isfile, join

**display_image**:
Draws bounding boxes around detected objects on an image and saves the annotated image.

*   image: PIL Image object representing the input image.
*   boxes: NumPy array of bounding box coordinates.
*   boxes: NumPy array of bounding box coordinates.
*   class_names: NumPy array of class names for detected objects.
*   scores: NumPy array of confidence scores for detected objects.
*   col_sel: Color selection for bounding boxes.
*   imgName: Name of the image file.
*   width_px, height_px: Width and height of the output image.

In [ ]:
def display_image(image, boxes, class_names, scores, col_sel, imgName, width_px, height_px):
    blank_image = Image.new("RGBA", (width_px, height_px), (255, 255, 255, 0))
    draw = ImageDraw.Draw(blank_image, 'RGBA')
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf", 75)

    for i in range(min(boxes.shape[0], 10)):  # Display up to 10 boxes
        if scores[i] >= 0.1:  # Minimum confidence score
            ymin, xmin, ymax, xmax = tuple(boxes[i])
            display_str = "{}: {}%".format(class_names[i].decode("ascii"), int(100 * scores[i]))

            color = get_color(display_str)

            if color == col_sel:
                draw_box(draw, (xmin, ymin, xmax, ymax), color, width_px, height_px)
                draw_text(draw, font, display_str, (xmin, ymin), color)

    file_path = f'layer/{col_sel}/img{imgName}.png'
    print("Display image", file_path)
    blank_image.save(file_path, format='png')
    print("Image saved to %s." % file_path)

**get_color**:
Determines the color for bounding boxes based on object classes.

*   display_str: String representing the class name.

In [ ]:
def get_color(display_str):
    color_groups = {
        "#006666": ["Tree", "Palm tree"],
        "#00B7B7": ["Vehicle", "vehicle", "Car", "Wheel", "Van", "Bicycle", "Boat"],
        "#013346": ["Water", "Pond"],
        "#08a294": ["Street"],
        "#00e59e": ["Building", "House", "Window", "Door", "Skyscraper"],
        "#ea6868": ["Person", "Clothing", "Woman", "Human", "Man", "Human face"],
        "#8CEE92": ["Street light"],
        "#036D5E": ["Billboard", "Traffic sign"],
    }
    for color, classes in color_groups.items():
        if any(c in display_str for c in classes):
            return color
    return "#ea3636"  # Default color

**draw_box**:
Draws a bounding box on an image.

*   draw: PIL ImageDraw object for drawing on the image.
*   box: Tuple representing bounding box coordinates.
*   color: Color code for the bounding box.
*   width_px, height_px: Width and height of the image.

In [ ]:
def draw_box(draw, box, color, width_px, height_px):
    im_width, im_height = width_px, height_px
    xmin, ymin, xmax, ymax = box
    left, right, top, bottom = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
    rgba_color = ImageColor.getrgb(color)
    draw.rectangle([(left, top), (right, bottom)], outline=rgba_color, width=3)

**draw_text**:
Draws text on an image.

*   draw: PIL ImageDraw object for drawing on the image.
*   font: PIL ImageFont object for specifying the font.
*   text: Text to be drawn.
*   position: Tuple representing the position to draw the text.
*   color: Color code for the text.

In [ ]:
def draw_text(draw, font, text, position, color):
    left, top = position
    rgba_color = ImageColor.getrgb(color)
    draw.text((left + 5, top - 75), text, fill="white", font=font, align="center")

**load_image**:
Loads and resizes an image.

*   path: Path to the image file.
*   new_width, new_height: Dimensions for resizing the image.

In [ ]:
def load_image(path, new_width, new_height):
    img = Image.open(path)
    img = ImageOps.fit(img, (new_width, new_height), Image.LANCZOS)
    return img

**run_detector**:
Runs object detection on an image and displays the results.

*   col_sel: Color selection for bounding boxes.
imgName: Name of the image file.
*   detector: TensorFlow Hub object detection model.
*   path: Path to the image file.

In [ ]:
def run_detector(col_sel, imgName, detector, path):
    img = load_image(path, width_px, height_px)
    converted_img = tf.image.convert_image_dtype(np.array(img), tf.float32)[tf.newaxis, ...]
    result = detector(converted_img)
    result = {key: value.numpy() for key, value in result.items()}
    display_image(img, result["detection_boxes"], result["detection_class_entities"],
                  result["detection_scores"], col_sel, imgName, width_px, height_px)
    return result

In [ ]:
# Module handle for object detection
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
detector = hub.load(module_handle).signatures['default']

# Set image dimensions
width_px, height_px = 800, 600

# Process images in a directory
mypath = "/content/gdrive/MyDrive/0_work/3_MIT/data_collection/18_segments/05_extra_frames/00_selected_frames/" #replace with your folder file path
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
color_layers = ["#006666", "#00B7B7", "#013346", "#08a294", "#00e59e", "#ea6868", "#8CEE92", "#036D5E"]
for index, imgName in enumerate(onlyfiles):
    print(imgName)
    image_path = join(mypath, imgName)
    result = run_detector(color_layers[index], imgName, detector, image_path)
